# Pryngles module

## Module base

Goals of the module:
- Future `base`
- Basic Classes and functions of the Class.
- It is based on existing infrastructure.

In [764]:
##HEADER
from pryngles import *
import unittest
#import rebound as rb
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [765]:
class Test(unittest.TestCase):
    from pryngles import _base

In [766]:
class Object(PrynglesCommon):
    """
    Global class of object
    """
    def _updateChilds(self,child=None):
        if 'childs' not in self.__dict__:
            self.childs=[]
        if child is not None:
            self.childs+=[child]
    def _updateParent(self,parent=None):
        if 'parent' not in self.__dict__:
            self.parent=parent
        elif parent is not None:
            self.parent=parent

In [767]:
def test_Object(self):
    obj=Object()
    obj._updateParent("parent")
    obj._updateChilds("child1")
    obj._updateChilds("child2")
    self.assertEqual([obj.parent],["parent"],True)
    self.assertEqual(obj.childs,["child1","child2"],True)
Test.test_Object=test_Object
#unittest.main(argv=['first-arg-is-ignored'],exit=False)

In [768]:
def addStar(self,
                 hash=None,
                 center=None,
                 m=1,R=1,
                 Prot=0.1,
                 N=1000,
             star=None
            ):
    """
    TO DEVELOPERS:
        This method is for the System class.  It is placed here because when a new parameter
        is added to the initializer of the addObject (being Object Star, Planet, etc.) class
        then the parameters of this ruotine should also change.
    """
    if star is not None:
        self._addObject("star","Star",star)
    else:
        self.stars+=[Star(hash,center,m,R,Prot,N)]
    self._updateSystem()
    
class Star(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 m=1,R=1,
                 Prot=0.1,
                 N=1000,
                ):
        #List of arguments: hash,center,m,r,Prot,N
        #Hash of the object
        self.hash=hash
        self.type="Star"

        #Center of the system (object)
        self.center=center
        if self.center is not None:
            self.center._updateChilds(self)
            self._updateParent(self.center)
        
        #Basic common properties
        self.m=m #mass [cu]
        self.R=R #radius [cu]

        #Dynamic parameters
        self.Prot=Prot

        #Sampling parameters
        self.N=N #number of spangles
        
        #Update properties
        self.updateObject(**self.__dict__)

    def updateObject(self,**props):
        self.__dict__.update(props)
        self._updateChilds()
        self._updateParent()

In [769]:
def addPlanet(self,
                 hash=None,
                 center=None,
                 m=1e-3,R=0.1,
                 Prot=0.01,
                 N=1000,
             planet=None
            ):
    if planet is not None:
        self._addObject("planet","Planet",planet)
    else:
        self.planets+=[Planet(hash,center,m,R,Prot,N)]
    self._updateSystem()
    
class Planet(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 m=1e-3,R=0.1,
                 Prot=0.01,
                 N=1000,
                ):
        #List of arguments: hash,center,m,r,Prot,N
        #Hash of the object
        self.hash=hash
        self.type="Planet"

        #Center of the system (object)
        if center is None:
            raise ValueError(f"You must provide a valid object for the center.  {center} provided")
        self.center=center
        self.center._updateChilds(self)
        self._updateParent(self.center)
        
        #Basic common properties
        self.m=m #mass [cu]
        self.R=R #radius [cu]

        #Dynamic parameters
        self.Prot=Prot

        #Sampling parameters
        self.N=N #number of spangles
        
        #Update properties
        self.updateObject(**self.__dict__)
        
    def updateObject(self,**props):
        self.__dict__.update(props)
        self._updateChilds()
        self._updateParent()

In [770]:
def addRing(self,
                 hash=None,
                 center=None,
                 fi=1.5,fe=2.5,
                 N=1000,
             ring=None
            ):
    if ring is not None:
        self._addObject("ring","Ring",ring)
    else:
        self.rings+=[Ring(hash,center,fi,fe,N)]
    self._updateSystem()
    
class Ring(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 fi=1.5,fe=2.5,
                 N=1000,
                ):
        #List of arguments: hash,center,fi,fe,N
        #Hash of the object
        self.hash=hash
        self.type="Ring"

        #Center of the system (object)
        if center is None:
            raise ValueError(f"You must provide a valid object for the center.  {center} provided")
        self.center=center
        self.center._updateChilds(self)
        self._updateParent(self.center)

        #Basic common properties
        self.fi=fi
        self.fe=fe

        #Sampling parameters
        self.N=N #number of spangles
        
        #Update properties
        self.updateObject(**self.__dict__)
        
    def updateObject(self,**props):
        self.__dict__.update(props)
        self._updateChilds()
        self._updateParent()
        
        #Derivative properties
        self.ri=self.fi*self.center.R
        self.re=self.fe*self.center.R

In [771]:
System_doc=\
"""
Creates a planetary system.

Examples:

    sys=System()
    sys=System(units=['msun','km','s'])
    sys=System(stars=dict(M=1,R=1))

Initialization (primary) attributes:

    units = ['msun','au','yr']: 
        Units used in calculations following the conventions and signs of rebound. List [3].
        
    stars = None: 
        Stars in the system.
    planets = None: 
        Planets in the system.
    rings = None: 
        Rings in the system.
           
        NOTE: Objects are provided either by description or as an object. Those attributes can be lists or 
              a single value.
    
    rebound = True:
        Set True if you want to simulte the orbit of objects (for instance if you want to calculate
        TTVs).  If False it will calculate orbits using Keplerian dynamics.

Other Public (secondary) attributes:

    N: Number of objects in the system.

Important private attributes:

    _sim: Rebound simulation object.
""";

In [772]:
class System(PrynglesCommon):
    
    def __init__(self,
                 units=['Msun','au','yr'],
                 stars=None,planets=None,rings=None,
                 rebound=True
                ):
        #Initialize rebound
        self.units=units

        #Add components
        self._list2Objects("stars","Star",stars)
        self._list2Objects("planets","Planet",planets)
        self._list2Objects("rings","Ring",rings)

        #Behavior
        self.rebound=True
        
        #Update system
        self._updateSystem()
    
    """
    #Freezer
    def addStar(self,star=None):
        self._addObject("star","Star",planet)
        self._updateSystem()
    
    def addPlanet(self,planet=None):
        self._addObject("planet","Planet",planet)
        self._updateSystem()
    
    def addRing(self,ring=None):
        self._addObject("ring","Ring",ring)
        self._updateSystem()
    """

    def _addObject(self,kind,objclass,obj):
        error=""
        try:
            obj.__dict__
            exec(f"self.{kind}s=self.{kind}s+[obj] if len(self.{kind}s)!=0 is not None else [obj]")
            cond=eval(f"obj.type!='{objclass}'")
            if cond:
                error=f"You cannot add a {objclass} with a {obj.type} object"
        except:
            #exec(f"self.{kind}s+=self.{kind}s+[{objclass}(**obj)] if self.{kind}s[0] is not None else [{objclass}(**{kind})]")
            exec(f"self.{kind}s+=self.{kind}s+[obj] if len(self.{kind}s)!=0 else [obj]")
        if error!="":
            exec(f"self.{kind}s.pop()")
            raise AssertionError(error)
        self._updateSystem()
        
    
    def _updateSystem(self):
        #Count components
        self.Nstars=self.Nplanets=self.Nrings=0
        for obj in "stars","planets","rings":
            exec(f"self.N{obj}=len(self.{obj})")
        self.N=self.Nstars+self.Nplanets+self.Nrings

    def _list2Objects(self,attr,kind,comps):
        if comps is None:
            exec(f"self.{attr}=[]")
        else:
            #Check if stars is a list
            try:comps[0]
            except:comps=[comps]
            exec(f"self.{attr}=[]")
            for comp in comps:
                try:
                    comp.__dict__
                    exec(f"self.{attr}+=[comp]")
                except:
                    exec(f"self.{attr}+=[{kind}(**comp)]")

System.addStar=addStar
System.addPlanet=addPlanet
System.addRing=addRing
System.__doc__=System_doc

--Test--

In [763]:
if __name__=="__main__":
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


--End--

## Debugging

Battery of test:

1. Individual object creation 

In [782]:
#Single
S=Star()
print(S)
#Parameters
S=Star(m=2)
print(S)
#With dictionary
star=dict(m=3.5,R=5.6)
S=Star(**star)
print(S)

#Single
P=Planet(center=S)
print(P)
#Parameters
P=Planet(R=2,center=S)
print(P)
#With dictionary
planet=dict(m=0.5,R=0.6)
P=Planet(**planet,center=S)
print(P)
print(S)

{'hash': None, 'type': 'Star', 'center': None, 'm': 1, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}
{'hash': None, 'type': 'Star', 'center': None, 'm': 2, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}
{'hash': None, 'type': 'Star', 'center': None, 'm': 3.5, 'R': 5.6, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}
{'hash': None, 'type': 'Planet', 'center': <__main__.Star object at 0x7fb9186db5b0>, 'parent': <__main__.Star object at 0x7fb9186db5b0>, 'm': 0.001, 'R': 0.1, 'Prot': 0.01, 'N': 1000, 'childs': []}
{'hash': None, 'type': 'Planet', 'center': <__main__.Star object at 0x7fb9186db5b0>, 'parent': <__main__.Star object at 0x7fb9186db5b0>, 'm': 0.001, 'R': 2, 'Prot': 0.01, 'N': 1000, 'childs': []}
{'hash': None, 'type': 'Planet', 'center': <__main__.Star object at 0x7fb9186db5b0>, 'parent': <__main__.Star object at 0x7fb9186db5b0>, 'm': 0.5, 'R': 0.6, 'Prot': 0.01, 'N': 1000, 'childs': []}
{'hash': None, 'type': 'Star', 'center': None, 'm': 3.5

2. System creation

In [784]:
#Default
sys=System()
print(sys)

{'units': ['Msun', 'au', 'yr'], 'stars': [], 'planets': [], 'rings': [], 'rebound': True, 'Nstars': 0, 'Nplanets': 0, 'Nrings': 0, 'N': 0}


In [744]:
sys.addStar(star=Star())

In [745]:
sys.addStar(star=Planet(center=sys.stars[0]))

AssertionError: You cannot add a Star with a Planet object

In [746]:
print(sys.stars)
print(sys.planets)
print(sys.rings)
print(sys.N)

[]
[]
1


In [747]:
sys.addPlanet(planet=Planet(center=sys.stars[0]))

In [748]:
print(sys.stars)
print(sys.planets)
print(sys.rings)
print(sys.N)

[]
2


In [749]:
sys.addPlanet(planet=Star())

AssertionError: You cannot add a Planet with a Star object

In [750]:
print(sys.stars)
print(sys.planets)
print(sys.rings)
print(sys.N)

[]
2


In [751]:
S=Star(m=1.89,R=0.1)

In [752]:
P=Planet(center=S,R=0.1)

In [753]:
S.childs,P.parent

([<__main__.Planet at 0x7fb9186e3460>], <__main__.Star at 0x7fb9186e3850>)

In [754]:
S=Star(m=1.89,R=0.1)
P=Planet(center=S,R=0.1)
R=Ring(center=P,fi=1)
R.updateObject(fi=2)
print(S)
print(P)
print(R)

{'hash': None, 'type': 'Star', 'center': None, 'm': 1.89, 'R': 0.1, 'Prot': 0.1, 'N': 1000, 'childs': [<__main__.Planet object at 0x7fb9186e3a00>], 'parent': None}
{'hash': None, 'type': 'Planet', 'center': <__main__.Star object at 0x7fb92bae23d0>, 'parent': <__main__.Star object at 0x7fb92bae23d0>, 'm': 0.001, 'R': 0.1, 'Prot': 0.01, 'N': 1000, 'childs': [<__main__.Ring object at 0x7fb92bae2a30>]}
{'hash': None, 'type': 'Ring', 'center': <__main__.Planet object at 0x7fb9186e3a00>, 'parent': <__main__.Planet object at 0x7fb9186e3a00>, 'fi': 2, 'fe': 2.5, 'N': 1000, 'childs': [], 'ri': 0.2, 're': 0.25}


In [755]:
S.childs

In [756]:
sys=System(stars=[S])
print(sys.stars[0])
sys=System(stars=[dict(m=2)])
print(sys.stars[0])

{'hash': None, 'type': 'Star', 'center': None, 'm': 1.89, 'R': 0.1, 'Prot': 0.1, 'N': 1000, 'childs': [<__main__.Planet object at 0x7fb9186e3a00>], 'parent': None}
{'hash': None, 'type': 'Star', 'center': None, 'm': 2, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}


In [757]:
sys=System()
print(sys.stars)
print(sys.planets)
print(sys.rings)
print(sys.N)

[]
[]
[]
0


In [758]:
sys.addStar(star=Star())

In [759]:
print(sys.stars[0])

{'hash': None, 'type': 'Star', 'center': None, 'm': 1, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}


In [760]:
sys.addStar(m=0.5)

In [761]:
print(sys.stars[1])

{'hash': None, 'type': 'Star', 'center': None, 'm': 0.5, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}


In [616]:
sys.addPlanet(planet=Star())

In [617]:
sys.planets

In [578]:
print(sys.stars[0])

{'hash': <__main__.Star object at 0x7fb949174f40>, 'type': 'Star', 'center': None, 'm': 1, 'R': 1, 'Prot': 0.1, 'N': 1000, 'childs': [], 'parent': None}


In [579]:
sys.stars

In [580]:
sys.addStar(Planet(center=sys.stars[0]))

In [526]:
sys.addPlanet(Star())

AssertionError: You cannot add a Planet with a Star object

In [524]:
sys.addStar(dict(m=2))

In [525]:
print(sys.stars)
print(sys.planets)
print(sys.rings)
print(sys.Nstars,sys.Nplanets,sys.Nrings,sys.N)

[<__main__.Star object at 0x7fb9387cb910>, <__main__.Planet object at 0x7fb9387cb820>, <__main__.Star object at 0x7fb9387cb910>, <__main__.Planet object at 0x7fb9387cb820>, <__main__.Star object at 0x7fb898060b50>]
[None]
5 1 0 6


In [381]:
print(sys.stars)

In [382]:
P1=Planet(center=S,hash="P1")
P2=Planet(center=S,hash="P2")

# Convert

In [60]:
%%javascript
IPython.notebook.kernel.execute('FILE=\"' + IPython.notebook.notebook_name + '\"')

<IPython.core.display.Javascript object>

In [61]:
!make -C .. DEVFILES=dev/{FILE} convert

Converting iPython Notebooks dev/pryngles-_template.ipynb...
Analysing file pryngles-_template.ipynb:
	Directory: src//pryngles
	Filename: _template
	Target object: src//pryngles/_template.py
	Converting from ipynb dev/pryngles-_template.ipynb to python src//pryngles/_template.py...
	Processing magic commands...
	Triming end...
	Triming test...
	Using as template src/.temp
Completed.


In [3]:
# Converted